In [1]:
%cd ..
%pwd

/Users/areshytko/repos/temp/productivity-dashboard


'/Users/areshytko/repos/temp/productivity-dashboard'

In [2]:
from dashboard.load import load
import dashboard.config as config 
from dashboard.gheets import ManualFlow

import json

In [3]:
flow = ManualFlow()
url = flow.get_url()
with open('notebooks/url.json', 'w') as wf:
    json.dump(url, wf)

In [ ]:
flow.put_code("4/1AY0e-g4MF6Ybonk-Qai_7F8K31zJ4cSUYy72GE_S4V2gvoLyGgfHywr2iyE")

In [4]:
creds = flow.get_google_token()

In [5]:
data = load(
    credentials=creds,
    pomodoros_spreadsheet_id=config.POMODOROS_SPREADSHEET_ID,
    pomodoros_range=config.POMODOROS_RANGE,
    activities_spreadsheet_id=config.ACTIVITIES_SPREADSHEET_ID,
    activities_range=config.ACTIVITIES_RANGE
)

In [6]:
data.df.head()

,Week,Date,Activity,Comment,Pomodoros,Planned,learn,do,career,green,red,personal,hobby,society,life,health
0,2,2021-01-06,Learn/System Design,None,2.0,4.0,True,False,True,True,False,False,False,False,False,False
1,2,2021-01-06,Learn/Golang,None,2.0,2.0,True,False,True,True,False,False,False,False,False,False
2,2,2021-01-06,Learn/Graph Algorithms,None,2.0,2.0,True,False,False,True,False,False,True,False,False,False
3,2,2021-01-06,Learn/Personal Development,None,1.0,1.0,True,False,False,False,False,True,False,False,False,False
4,2,2021-01-06,Do/Algorithms,None,NaN,2.0,False,True,True,True,False,False,False,False,False,False


## Process

In [19]:
from typing import Optional
import pandas as pd
import numpy as np
from dashboard.load import PomodorosProcessed, ActivitiesCatalog
from dashboard.process import WeeklyStats


def complete_rate(df: pd.DataFrame) -> pd.Series:
    return df['done'] / df['planned']

def do_learn_ratio(df: pd.DataFrame) -> Optional[float]:
    do = df.loc[df['do'], 'Pomodoros'].sum()
    learn = df.loc[df['learn'], 'Pomodoros'].sum()
    
    if do == 0 and learn == 0:
        return None
    
    return 2 * do / (do + learn)

def red_green_ratio(df: pd.DataFrame) -> Optional[float]:
    green = df.loc[df['green'], 'Pomodoros'].sum()
    red = df.loc[df['red'], 'Pomodoros'].sum()
    
    if green == 0 and red == 0:
        return None
    
    return 2 * green / (green + red)

def mse(p, q):
    return np.sqrt(np.mean(np.sum(np.power((np.array(p) - np.array(q)), 2))))

LIFE_DOMAINS = ['life', 'personal', 'career', 'hobby', 'society', 'health']

def balance_coef(df: pd.DataFrame, balanced_distr: pd.Series) -> float:
    assert np.all(balanced_distr.index == LIFE_DOMAINS)
    all_pomodoros = df.Pomodoros.sum()
    actual_dist = [df.loc[df[col], 'Pomodoros'].sum() / all_pomodoros for col in LIFE_DOMAINS]

    return mse(balanced_distr, actual_dist)

In [8]:
x = pd.DataFrame({
    'Pomodoros': [1, 1, 1, 1, 1],
    'life': [False, False, False, False, True],
    'personal': [True, True, True, False, False],
    'career': [False, False, False, False, False],
    'hobby': [False, False, False, False, False],
    'society': [False, False, False, True, False],
    'health': [False, False, False, False, False]
})

expected_zero = pd.Series([0.2, 0.6, 0, 0, 0.2, 0], index=LIFE_DOMAINS)
expected_small = pd.Series([0.2, 0.5, 0.1, 0, 0.2, 0], index=LIFE_DOMAINS)
expected_big = pd.Series([0, 0, 0, 1, 0, 0], index=LIFE_DOMAINS)

balance_coef(x, expected_zero), balance_coef(x, expected_small), balance_coef(x, expected_big)

(0.0, 0.1414213562373095, 1.2)

In [10]:
def process_week(week, df):

    result = {
        'from_date': df.Date.min(),
        'to_date': df.Date.max(),
        'done': df.Pomodoros.sum(),
        'planned': df.Planned.sum(),
        'avg_done': df.Pomodoros.mean(),
        'avg_planned': df.Planned.mean(),
        'do_learn_ratio': do_learn_ratio(df),
        'red_green_ratio': red_green_ratio(df),
        'balance_coef': balance_coef(df, pd.Series([0.2, 0.6, 0, 0, 0.2, 0], index=LIFE_DOMAINS))
    }
    return pd.DataFrame(result, index=[week])

result = pd.concat([process_week(week, df) for week, df in data.df.groupby('Week')])
result['complete_rate'] = complete_rate(result)
result

,from_date,to_date,done,planned,avg_done,avg_planned,do_learn_ratio,red_green_ratio,balance_coef,complete_rate
2,2021-01-06,2021-01-10,17.0,57.0,1.545455,1.900000,0.235294,2.000000,0.627402,0.298246
3,2021-01-11,2021-01-17,100.0,120.0,9.090909,8.571429,1.878788,1.979167,1.146822,0.833333
4,2021-01-18,2021-01-24,20.0,58.0,2.222222,2.071429,0.000000,2.000000,0.989949,0.344828
5,2021-01-25,2021-01-31,39.0,58.0,1.772727,1.450000,0.324324,1.882353,0.969705,0.672414
6,2021-02-01,2021-02-14,69.0,106.0,2.300000,1.859649,0.925926,1.612903,0.887690,0.650943
7,2021-02-15,2021-02-21,3.0,30.0,1.500000,1.764706,NaN,0.000000,0.742369,0.100000
10,2021-03-08,2021-03-14,25.0,51.0,2.777778,2.125000,0.857143,1.680000,0.890842,0.490196
11,2021-03-15,2021-03-21,23.0,39.0,1.916667,2.052632,0.782609,2.000000,0.855495,0.589744


In [18]:
from dashboard.process import WeeklyStats

weekly_stats = WeeklyStats.convert(result.reset_index().rename(columns={'index':'Week'}))
weekly_stats.df

,Week,from_date,to_date,done,planned,avg_done,avg_planned,do_learn_ratio,red_green_ratio,balance_coef,complete_rate
0,2,2021-01-06,2021-01-10,17.0,57.0,1.545455,1.900000,0.235294,2.000000,0.627402,0.298246
1,3,2021-01-11,2021-01-17,100.0,120.0,9.090909,8.571429,1.878788,1.979167,1.146822,0.833333
2,4,2021-01-18,2021-01-24,20.0,58.0,2.222222,2.071429,0.000000,2.000000,0.989949,0.344828
3,5,2021-01-25,2021-01-31,39.0,58.0,1.772727,1.450000,0.324324,1.882353,0.969705,0.672414
4,6,2021-02-01,2021-02-14,69.0,106.0,2.300000,1.859649,0.925926,1.612903,0.887690,0.650943
5,7,2021-02-15,2021-02-21,3.0,30.0,1.500000,1.764706,NaN,0.000000,0.742369,0.100000
6,10,2021-03-08,2021-03-14,25.0,51.0,2.777778,2.125000,0.857143,1.680000,0.890842,0.490196
7,11,2021-03-15,2021-03-21,23.0,39.0,1.916667,2.052632,0.782609,2.000000,0.855495,0.589744


In [9]:
from dashboard.load import PomodorosProcessed, ActivitiesCatalog
from dashboard.process import WeeklyStats

def compute_weekly_stats(data: PomodorosProcessed) -> WeeklyStats:
    pass